### Implement Sigmoid

Following implements $sigmoid(x)\:=\: \frac{1}{1 + e^{-x}}$

First create a Graph, which is the recipe for the sequence of operations to perform

In [1]:
import tensorflow as tf
import numpy as np

#Can create a Graph as follows
#graph = tf.Graph()
#or
#graph = tf.get_default_graph()

Create tensors
- x: Placeholder, the variables that we will give at runtime
- W and b: mutable tensors, they change with time, say in each iteration, variables need an initializer
- h: imutable tensor, produces as a result of some operations on other tensors


In [2]:
x = tf.placeholder(shape = [None, 10], name = 'x', dtype = tf.float32)
W = tf.Variable(tf.random_uniform(shape = [10, 5], dtype = tf.float32, minval = -0.1, maxval = 0.1), name = 'W')
b = tf.Variable(tf.zeros(shape = [5], dtype = tf.float32), name = 'b')
h = tf.nn.sigmoid(tf.matmul(x, W) + b)
print('shape of h is', h.shape)
init = tf.global_variables_initializer()
np.random.seed(1234)
with tf.Session() as session:
    session.run(init)
    h_eval = session.run(h, feed_dict = {x: np.random.rand(1, 10)})

print('h_eval is', h_eval)

shape of h is (?, 5)
h_eval is [[ 0.52049112  0.49635988  0.48334688  0.49741691  0.48985034]]



Placeholders can also be replaced by preloaded constants where the values do not depend on runtime as follows, note that we dont need ``feed_dict`` in this case as the input is constant


In [3]:
c = tf.constant(value = [[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]], dtype= tf.float32)
W_c = tf.Variable(tf.random_uniform(shape = [10, 5], dtype = tf.float32, minval = -0.1, maxval = 0.1), name = 'W')
b_c = tf.Variable(tf.zeros(shape = [5], dtype = tf.float32), name = 'b')
h_c = tf.nn.sigmoid(tf.matmul(c, W) + b)
print('shape if h is', h_c.shape)
with tf.Session() as session:
    tf.global_variables_initializer().run()
    h_eval = session.run(h_c)

print('h_eval is', h_eval)

shape if h is (1, 5)
h_eval is [[ 0.15896994  0.47830001  0.7322498   0.34032491  0.73249787]]


### Building pipeline

We will now source the data from 3 text files files each containing 5 lines with each line being 10 comma seperated float values. First, we will create the files if they don't already exist with sample data and then build a pipeline to read the data from files all the way to loading them into tensors.

In [4]:
def create_sample_if_not_exist(filename, numrows = 5, numcols = 10, seed = None):
    import numpy as np
    import os
    if os.path.exists(filename):
        print('File', filename, 'exists, not creating one')
    else:
        print('Creating', filename)
        if seed is not None:
            np.random.seed(seed)
    
        with open(filename, 'w') as f:
            for _ in range(numrows):
                f.write(",".join(np.random.rand(numcols).astype(str).tolist()) + "\n")

In [5]:
filenames = ['text%d.txt'%i for i in range(1, 4)]
for filename in filenames:
    create_sample_if_not_exist(filename)

File text1.txt exists, not creating one
File text2.txt exists, not creating one
File text3.txt exists, not creating one


We will create a string input producer that will peduce tensors holding the string values held in filenames. We shuffle the input and bound the queue to size 3.  

In [6]:
filename_queue = tf.train.string_input_producer(filenames, capacity = 3, shuffle = True, name = 'str_inp_prod')

Once the queue is created, we will initialize a ``TextLineReader`` which will read a line from the text. The read operation returns the values as a key and a value. The key is the name of the file and the value is one line from this file

In [7]:
reader = tf.TextLineReader()
key, value = reader.read(filename_queue, name = 'text_line_op')

#Incase of bad line
default_record = [[-1.0]] * 10

Decode csv to read 10 values (which we expect in our csv file). These would be 10 independent tensors, we want these 10 values to become our feature vector

In [8]:
col1, col2, col3, col4, col5, col6, col7, col8, col9, col10 = tf.decode_csv(value, record_defaults = default_record)
feature = tf.stack([col1, col2, col3, col4, col5, col6, col7, col8, col9, col10])


The variable ``features`` above represents one tensor of size (10, ), representing a single training sample from the input files. We need to read multiple records in batches. Following line allows us to read multiple tensors as a batch. The parameters ``batch_size`` is the mini batch size, capacity is the capacity of the queue containing items, the value ``min_after_dequeue`` represents the number of values to retain in the queue after the dequee operation of the elements in batch. The ``num_threads`` decides how many threads will generate data in parallel.

The below setup allows tensor flow read batches of data abstracting the source of real data. The source data will be accessed slowly and can be parallelized to retrieve data asynchronously to the whole training process. The tensor below is our input to the training process

In [9]:
x = tf.train.shuffle_batch([feature], batch_size = 3, capacity = 5, min_after_dequeue = 1, num_threads = 2)
print('shape of x is expected to be (<batch_size>, <feature_dimension>) and is', x.shape)

shape of x is expected to be (<batch_size>, <feature_dimension>) and is (3, 10)


In [10]:
#Define the sigmoid operation again

W = tf.Variable(tf.random_uniform(shape = (10, 5), minval = -0.1, maxval = 0.1, dtype = tf.float32), name = 'W')
b = tf.Variable(tf.zeros(shape = (5, )), dtype = tf.float32, name = 'b')
h = tf.nn.sigmoid(tf.matmul(x, W) + b)
with tf.Session() as session:
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord = coord, sess = session)
    tf.global_variables_initializer().run()
    
    for step in range(5):
        x_eval, h_eval = session.run([x, h])
        print('================ Step %d ================'%step)
        print('Evaluated data (x)')
        print(x_eval)
        print('Evaluated data (h)')
        print(h_eval)
    
    #Request all threads to stop processing
    coord.request_stop()
    #Wait for all threads to gracefully stop, not needed in a notebook environment
    coord.join(threads)

================ Step 0 ================
Evaluated data (x)
[[ 0.72665846  0.90008783  0.77916378  0.59915477  0.29112524  0.15139526
   0.33517465  0.65755177  0.07334255  0.0550064 ]
 [ 0.32570741  0.19361869  0.45781165  0.92040259  0.87906915  0.25261575
   0.34800878  0.18258873  0.90179604  0.70652819]
 [ 0.00934857  0.90064859  0.97724146  0.55689466  0.08477385  0.33300248
   0.72842866  0.14243537  0.55246896  0.27304325]]
Evaluated data (h)
[[ 0.49464947  0.52583575  0.50423062  0.49881858  0.50255448]
 [ 0.45626062  0.53781515  0.50380903  0.49873108  0.51279169]
 [ 0.48437965  0.51267284  0.50956738  0.50125998  0.52063322]]
================ Step 1 ================
Evaluated data (x)
[[ 0.73852307  0.58730364  0.47163254  0.10712682  0.22921857  0.89996517
   0.41675353  0.53585166  0.00620852  0.30064172]
 [ 0.97449511  0.6677869   0.25565329  0.1083115   0.77618074  0.78247797
   0.76160389  0.91440314  0.6586228   0.5683676 ]
 [ 0.28525096  0.62491673  0.4780938   0.1956